In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
usharengaraju_agriseshat_agriculture_dataset_path = kagglehub.dataset_download('usharengaraju/agriseshat-agriculture-dataset')
sathyanarayanrao89_soil_moisture_data_from_field_scale_sensor_network_path = kagglehub.dataset_download('sathyanarayanrao89/soil-moisture-data-from-field-scale-sensor-network')

print('Data source import complete.')


100%|██████████| 68.7k/68.7k [00:00<00:00, 44.3MB/s]

Extracting files...


100%|██████████| 14.5M/14.5M [00:00<00:00, 158MB/s]

Extracting files...


Data source import complete.


## Table of Contents

* [1. Introduction to the Notebook](#1)
* [2. Area of Interest: Berambadi](#2)
* [3. Google Earth Initialization](#3)
* [4. Sentinel-1 Imagery](#4)
* [5. NDVI from Sentinel-2 Imagery](#5)
* [6. NDVI from LANDSAT8 Imagery](#6)
* [7. Backscatter time series](#7)


<a id="1"></a>
## 1. Introduction to the Notebook

<font size="3">  

Google earth engine is a platform for Geospatial data analysis from Google. It has data from many different satellites including the Sentinel from European Space Agency. I mostly learnt syntax and imported several code snippets from google developer site:  
[https://developers.google.com/earth-engine/tutorials/community/explore](http://)

Here I am trying to use google earth engine to retrieve sentinel-1 data of Berambadi water shed region.

</p>

### import libraries

In [2]:
import folium
import ee
import matplotlib.pyplot as plt
import seaborn as sns
import IPython.display as disp
from branca.element import Figure
import numpy as np

<a id="2"></a>
## 2. Area of interest: Berambadi watershed

<font size="3">  
    
Berambadi watershed is an area in Chamarajanagar district of Karnataka India. The climate of the region is semi-arid and situated near
rainforest and tiger reserve. Many renowned soil-scientists and environmental scientists from around the world have doen thier work
on climate change, hydrology and agronomy in this region. Due to over exploitation of ground water, this region is deemed as critically
endagered in terms of ground water recharge. Hence our interest to learn google earth through Gundlupet.


#### Lets briefly view the region using Folium

In [3]:
fig1 = Figure(width=550,height=350)

map1 = folium.Map(location=[11.777968, 76.597909],zoom_start=12, width=550,height=350);
fig1.add_child(map1)
# lets mark Gundlupet bus_stand on the map
folium.Marker(location=[11.777968, 76.597909],popup='Default popup Marker1',tooltip='Click here to see Popup').add_to(map1)
display(fig1)

#### Mark the Area of Interest (AOI)

<font size="3">
    
Here I draw a polygon of Berambadi watershed region using GeoJSON file.
I copied the GeoJSON file by marking the AOI by hand in the website: [https://geojson.io/](http://)
Here I am marking an area surrouding the Berambadi watershed region arbitaririly in the above link and copy the coordinate geoJSON file below:

In [4]:
watershed = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              76.60413366762856,
              11.78151054870402
            ],
            [
              76.57344831024034,
              11.794260585249859
            ],
            [
              76.56196889596544,
              11.78161860234809
            ],
            [
              76.55622918882744,
              11.756549018691018
            ],
            [
              76.57377944719059,
              11.750713529259215
            ],
            [
              76.60567897339666,
              11.748119938711639
            ],
            [
              76.61450929206984,
              11.769840505010293
            ],
            [
              76.63261144534937,
              11.775243364615974
            ],
            [
              76.65866088543544,
              11.787885640990154
            ],
            [
              76.65369383118195,
              11.796853740543014
            ],
            [
              76.6294104548312,
              11.810359361287453
            ],
            [
              76.60413366762856,
              11.78151054870402
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}


In [5]:
folium.GeoJson(data=watershed,style_function=lambda x: {'fillColor': 'orange'}).add_to(map1);
display(fig1)

<a id="2"></a>
## 3. Google Earth Initialization

<font size="3">
We need to use Earth Engine Authenticate and Initialize functions before using its other functionality.
More on these functions: [https://developers.google.com/earth-engine/apidocs/ee-authenticate](http://).

In [6]:
ee.Authenticate()

In [8]:
ee.Initialize(project='YOUR_PROJECT_ID')

EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

<font size="3">
I copied the GeoJSON file by marking the AOI by hand in the website: [https://geojson.io/](http://)
Here I am marking an area surrouding the Berambadi watershed region arbitaririly in the above link and
copy the coordinate geoJSON file below:


In [9]:
coords = watershed['features'][0]['geometry']['coordinates']
aoi = ee.Geometry.Polygon(coords)

EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

<a id="4"></a>
## 4. Sentinel-1 Imagery

#### Available polarization on Gundlupet region

In [10]:
image = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(aoi).filterDate('2022-08-01', '2022-12-31').first().clip(aoi);
image.bandNames().getInfo()

EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

#### Visualizing VH backscatter on area of interest

In [11]:
url = image.select('VV').getThumbURL({'min': -20, 'max': 0})
disp.Image(url=url, width=800)

NameError: name 'image' is not defined

<font size="3">  

I burrow the function from the below link which enables me to plot satellite backscatter on top of folium map.
[https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2](http://)
    
</p>

In [12]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

#### Visualizing VV backscatter on the map of Folium

In [14]:
# Create the map object.
fig2 = Figure(width=550,height=550)
image = ee.Image(image.select('VV'))
map2 = folium.Map(location=[11.777968, 76.597909],zoom_start=13, width=550,height=350);
fig2.add_child(map2)

# Add the S1 rgb composite to the map object.
map2.add_ee_layer(image,{'min': -20, 'max': 0}, 'FFA')

# Display the map.
display(map2)

NameError: name 'image' is not defined

<a id="5"></a>
## 5. NDVI from Sentinel-2 Imagery

<font size="3">
    
Normalized Difference Vegetation Index (NDVI) is a remote sensing index that is commonly used to measure the amount of vegetation in an area. NDVI is calculated from the red and near-infrared (NIR) bands of a satellite or aerial image, and it ranges from -1 to 1.

NDVI is calculated by the following formula:

NDVI = (NIR - Red) / (NIR + Red)

Where NIR is the reflectance of the near-infrared band and Red is the reflectance of the red band.

When NDVI value is close to 1, it indicates that the area is covered by healthy vegetation. NDVI values between 0 and 0.2 indicate barren or sparse vegetation, while NDVI values between 0.2 and 0.5 indicate dry vegetation. NDVI values between 0.5 and 0.8 indicate healthy vegetation and NDVI values above 0.8 indicate dense vegetation.

NDVI can be used to monitor crop growth, detect drought, map land cover, and map vegetation health. NDVI is a widely used index in agriculture, and it is a good indicator of plant health, biomass, and leaf area index. NDVI is also used to monitor land use changes, detect deforestation and other changes in land cover.

<font size="3">
We will plot NDVI using sentinel-2 data

In [15]:
ee.Initialize()

img_coll = ee.ImageCollection('COPERNICUS/S2_SR')
# Define the date range
date_range = ee.DateRange('2022-01-01', '2022-12-31')

img_coll = img_coll.filterBounds(aoi).filterDate(date_range)

# Get the first image from the filtered collection
img = img_coll.first()

# Define the red and NIR bands
red = ee.Image(img.select('B4'))
nir = ee.Image(img.select('B8'))

# Calculate NDVI
ndvi = (nir.subtract(red)).divide(nir.add(red))

EEException: ee.Initialize: no project found. Call with project= or see http://goo.gle/ee-auth.

In [16]:
# Visualize the NDVI image on a map using Folium
ndvi_vis = {'min': -1, 'max': 1, 'palette': ['#00ff00', '#ffff00', '#ff0000']}
# Create the map object.
fig3 = Figure(width=550,height=550)
map3 = folium.Map(location=[11.777968, 76.597909],zoom_start=11, width=550,height=350);
fig3.add_child(map3)

# Add the S1 rgb composite to the map object.
map3.add_ee_layer(ndvi,ndvi_vis, 'FFA')
# Display the map.
display(map3)

NameError: name 'ndvi' is not defined

<a id="6"></a>
## 6. NDVI histogram from LANDSAT Imagery

<font size="3">

Landsat satellites carry multispectral sensors that capture images of the Earth in various spectral bands, including visible, near-infrared, and thermal infrared. The data collected by the Landsat satellites is used for a variety of applications, including agriculture and forestry. Here we will use it to plot histogram of NDVI index.

In [17]:
# Define the region of interest as a geometry.
region = ee.Geometry.Polygon(watershed['features'][0]['geometry']['coordinates'][0])
# Filter Landsat 8 images to only include the most recent.
collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_TOA') \
    .filterBounds(region) \
    .sort('system:time_start', False)

# Get the most recent image.
image = ee.Image(collection.first())

# Compute NDVI for the specified region.
ndvi = image.normalizedDifference(['B5', 'B4']).clip(region)

# Convert the image to an array so it can be plotted.
ndvi_array = ndvi.reduceRegion(
  reducer=ee.Reducer.toList(),
  geometry=region,
  maxPixels=1e10
).get('nd').getInfo()

# Plot the histogram of NDVI values.
import matplotlib.pyplot as plt
plt.hist(ndvi_array, bins=100)
plt.xlabel('NDVI Value')
plt.ylabel('Frequency')
plt.show()

EEException: Earth Engine client library not initialized. See http://goo.gle/ee-auth.

<a id="7"></a>
## 7. Backscatter time series

<font size="3">
VV and VH backscatter polarization refer to the polarization of the radar waves that are transmitted and received by a satellite sensor. In the case of VV polarization, the electric field of the radar wave is oriented vertically (along the vertical axis), while in the case of VH polarization, the electric field is oriented horizontally (along the horizontal axis).

A time series of backscatter refers to a series of measurements of backscatter coefficients at different points in time. The backscatter coefficient is a measure of how much of the radar energy is reflected back to the sensor.

A time series of backscatter provides important information about the evolution of the surface features over time. For example, changes in the backscatter coefficients can indicate changes in the surface roughness, vegetation, soil moisture, and other geophysical properties. These changes can provide valuable information for monitoring the environment, natural resources, and the impact of human activities on the earth's surface.

In [ ]:
import datetime
# Define the location of interest.
location = ee.Geometry.Point(76.3661, 12.4219).buffer(10000).bounds();

sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD')\
.filterBounds(location).filterDate('2021-08-01', '2022-12-31')\
.sort('system:time_start', False);

# Print the number of images in the filtered collection.
print('Number of images:', sentinel1.size().getInfo())



In [ ]:
# Get the dates of the images in the filtered collection
dates = sentinel1.aggregate_array("system:time_start").getInfo()

# Convert the dates from Unix time to human-readable format
readable_dates = [datetime.datetime.fromtimestamp(int(d) / 1000).strftime("%Y-%m-%d") for d in dates]

print("Available dates:", readable_dates)

In [ ]:
image1 = ee.Image(sentinel1.first())

value = image1.select('VV').reduceRegion(ee.Reducer.mean(), location, 30).getInfo()

value['VV']

In [ ]:
sentinel1.size().getInfo()

In [ ]:
images = sentinel1.toList(sentinel1.size().getInfo())
time_series = [];
# Loop through each image in the 140 images
for i in range(sentinel1.size().getInfo()):
    # Get the date of the selected image
    image = ee.Image(images.get(i))
    date = image.date().format("YYYY-MM-dd").getInfo()
    value = image.reduceRegion(ee.Reducer.mean(), location, 30).getInfo();
    time_series.append([date,value['VV'],value['VH']]);

In [ ]:
import pandas as pd
df = pd.DataFrame(time_series, columns=["date", "VV", "VH"])


In [ ]:
df

In [ ]:
fig1, ax = plt.subplots(1,1,dpi=100,figsize=(12, 5))
ax.plot(df['date'][0:50],df['VV'][0:50],label='VV');
ax.plot(df['date'][0:50],df['VH'][0:50],'r',label='VH');
ax.tick_params(axis='x', labelrotation = 90)
ax.set_title('VH,VV time series');
ax.set_xlabel('date')
ax.set_ylabel('backscatter [db]');
ax.legend()


<font size="3">
In conclusion, we have successfully plotted a time series of backscatter from Sentinel-1 satellite data.     